In [3]:
pip install --upgrade google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.46.0
    Uninstalling google-genai-1.46.0:
      Successfully uninstalled google-genai-1.46.0


In [1]:
# export GOOGLE_CLOUD_API_KEY="YOUR_API_KEY"


from google import genai
from google.genai import types
import base64
import os

def generate():
  client = genai.Client(
      vertexai=True,
      api_key=os.environ.get("GOOGLE_CLOUD_API_KEY"),
  )

  msg2_text1 = types.Part.from_text(text="""**Assessing the Constraints**

I'm currently focused on the limitations of the query. I've broken down the user's request and the imposed restrictions. The key here is to stay within the provided context. I'm noting the constraints to formulate a focused response.


**Evaluating Source Material**

I've reviewed the source material and noted that the user's question, \"What is snow?\", isn't directly answered. The sources mainly detail the ADS, its function, and related operational aspects. I'm strategizing how to address the user's implied informational need while staying within the provided context. My focus is on synthesizing relevant information from the sources.


**Defining the Response**

I've formulated a response based on the analysis. I considered the user's intent to learn \"something about snow.\" I've used the source material to outline the Alaska Department of Snow (ADS). I've focused on the ADS's function, vehicles, and the absence of a scientific definition of snow within the provided text.


**Structuring the Answer**

I'm now focusing on generating a cohesive response. I've synthesized information from the ADS sources, avoiding an attempt to define \"snow\" itself. I've prioritized the most pertinent details regarding the user's implicit information need. I'm focusing on organizing this into a clear, concise statement.""")
  msg2_text2 = types.Part.from_text(text="""Based on the provided sources, there is no definition of what snow is. The provided documents only contain information regarding the Alaska Department of Snow, which coordinates snow removal, publishes snowfall statistics, and operates specialized vehicles like snowplows to manage winter conditions.""")

  model = "gemini-3-pro-preview"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""can you tell me something about snow?""")
      ]
    ),
    types.Content(
      role="model",
      parts=[
        msg2_text1,
        msg2_text2
      ]
    ),
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""Can you tell me something about the alaska dept of snow?""")
      ]
    ),
  ]
  tools = [
    types.Tool(
      retrieval=types.Retrieval(
        vertex_rag_store=types.VertexRagStore(
          rag_resources=[
            types.VertexRagStoreRagResource(
              rag_corpus="projects/720196750972/locations/us-east1/ragCorpora/4749045807062188032"
            )
          ],
        )
      )
    )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    tools = tools,
    thinking_config=types.ThinkingConfig(
      thinking_level="HIGH",
    ),
  )

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    if not chunk.candidates or not chunk.candidates[0].content or not chunk.candidates[0].content.parts:
        continue
    print(chunk.text, end="")

generate()

The Alaska Department of Snow (ADS) was established in 1959. Its mission is to ensure safe travel and infrastructure continuity by coordinating snow removal services across the state's 650,000 square miles.

Additionally, the ADS handles controlled avalanche mitigation in mountainous areas and coordinates budget allocations based on factors such as snowfall averages and historical costs. Job openings for the department are listed on the official State of Alaska jobs website.